In [1]:
from openml import tasks

In [2]:
oml_task_diabetes = tasks.get_task(37)

In [67]:
X, y, categorical_indicator, attribute_names = oml_task_diabetes.get_dataset().get_data()

In [71]:
# for some reason X already includes class and y is None
# that's okay -> makes nested CV easier to handle, split in folds
Xy = X.copy()
Xy

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,tested_positive
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,tested_negative
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,tested_positive
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,tested_negative
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,tested_positive
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,tested_negative
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,tested_negative
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,tested_negative
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,tested_positive


In [72]:
categorical_indicator

[False, False, False, False, False, False, False, False, True]

In [73]:
import sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold

In [98]:
from benchmark import NeuralNetwork

In [ ]:
# outer
data_train_test, data_val = train_test_split(
    Xy,
    train_size=2/3,
    shuffle=True,
    stratify=Xy.loc[:, 'class']
)

# reset indices as StratifiedKFold assumes 0-(n-1) index
data_train_test = data_train_test.reset_index(drop=True)
data_val = data_val.reset_index(drop=True)

# inner
cv_inner = StratifiedKFold(
    n_splits=5,
    shuffle=False  # TODO: set to True
)

for i, (indices_train, indices_test) in enumerate(cv_inner.split(
    X=data_train_test,  # loc[:, data_train_test.columns != 'class']
    y=data_train_test.loc[:, 'class']
)):
    #print(f'fold {i + 1} / {cv_inner.get_n_splits()}')
    data_train = data_train_test.loc[indices_train, :]
    data_train_X = data_train.loc[:, data_train.columns != 'class']
    data_train_y = data_train.loc[:, 'class']
    data_test = data_train_test.loc[indices_test, :]
    data_test_X = data_test.loc[:, data_test.columns != 'class']
    data_test_y = data_test.loc[:, 'class']

    '''
    in each inner fold, run EAGGA + find an optimal pareto front of configurations for this specific fold
    then in the outer fold (holdout), compare each fold-specific optimal pareto front + select optimum
    '''

    # hp config loop starts here
    hp_configs = {
        'total_layers': [3],
        'nodes_per_hidden_layer': [3]
    }
    performance = {
        'auc': list(),
        'ni': list(),
        'nf': list(),
        'nnm': list()
    }
    while(True):
        nn = NeuralNetwork(
            input_size=len(data_train_X.columns),
            output_size=1,  # we only use binary datasets
            total_layers=hp_configs['total_layers'][-1],
            nodes_per_hidden_layer=hp_configs['nodes_per_hidden_layer'][-1]
        )
        print(nn)

        # TODO: implement training loop

        # TODO: evaluate performance + add to auc, NI, NF, NNM

        # TODO: EA on hp_total_layers + hp_nodes
        # TODO: implement group structure

        # until stopping criterion is met
        # TODO: define stopping criterion
        break

NeuralNetwork(
  (net): Sequential(
    (0): Linear(in_features=8, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3): ReLU()
    (4): Linear(in_features=3, out_features=1, bias=True)
    (5): Softmax(dim=None)
  )
)
NeuralNetwork(
  (net): Sequential(
    (0): Linear(in_features=8, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3): ReLU()
    (4): Linear(in_features=3, out_features=1, bias=True)
    (5): Softmax(dim=None)
  )
)
NeuralNetwork(
  (net): Sequential(
    (0): Linear(in_features=8, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3): ReLU()
    (4): Linear(in_features=3, out_features=1, bias=True)
    (5): Softmax(dim=None)
  )
)
NeuralNetwork(
  (net): Sequential(
    (0): Linear(in_features=8, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3)